A `delegate` is an `object` that knows `how to call a method`.

it defines the method’s `return type` and its `parameter types`.

In [2]:
//The following defines a delegate type called Transformer:

delegate int Transformer (int x);

int Square (int x) { return x * x; }
int MultipleBy2(int x) { return x * 2;}

Transformer t1 = Square;
Transformer t2 = MultipleBy2;

int answer1 = t1(2); //4 //Invoke delegate
int asnwer2 = t2(3); //6 //Invoke delegate

Console.WriteLine(answer1);

Console.WriteLine(asnwer2);

Transformer t3 = Square;
//in fact compelete code is 
Transformer t = new Transformer (Square);

//The expression
t(3);
//is shorthand for:
t.Invoke(3)

4
6


9

### Writing Plug-In Methods with Delegates

In [3]:
//A delegate variable is assigned a method at runtime. 
//This is useful for writing plug-in methods.

delegate int Transformer (int x);

int Square (int x) => x * x;
int Cube (int x) => x * x * x;

void Transform (int[] values, Transformer t)
{
    for (int i = 0; i < values.Length; i++)
    values[i] = t (values[i]);
}

int[] values = { 1, 2, 3 };
Transform (values, Square); 

foreach (int i in values)
    Console.Write (i + " "); 

1 4 9 

### Instance and Static Method Targets

In [ ]:
//A delegate’s target method can be a local, static, or instance method.

delegate int Transformer (int x);

class Test { public static int Square (int x) => x * x; }

Transformer t = Test.Square;
Console.WriteLine (t(10)); // 100

In [4]:
delegate int Transformer (int x);
class Test { int y = 10; public int Square (int x) => x * x * y; }

Test test = new Test();
Transformer t = test.Square;
Console.WriteLine (t(10));

100


#### ***Important***

When an `instance method` is `assigned` to a ***delegate object***, the latter maintains a reference `not only to the method` but also `to the instance` to which the method belongs.

The `System.Delegate` class’s `Target` property represents this instance 

In [5]:
public delegate void ProgressReporter (int percentComplete);

class MyReporter
{
    public string Prefix = "";
    public void ReportProgress (int percentComplete)
        => Console.WriteLine (Prefix + percentComplete);
}

MyReporter r = new MyReporter();
r.Prefix = "%Complete: ";
ProgressReporter p = r.ReportProgress;
p(99); // %Complete: 99
Console.WriteLine (p.Target == r); // True
Console.WriteLine (p.Method); // Void ReportProgress(Int32)
r.Prefix = "";
p(99); // 99

%Complete: 99
True
Void ReportProgress(Int32)
99


Because the `instance` is `stored` in the delegate’s property, its `lifetime` is
Target `extended` to (at least as long as) the `delegate’s lifetime`.

### Multicast Delegates

This means that a `delegate instance` can reference not just a single target method but also a `list of target methods`. The `+` and `+=` operators combine delegate instances

`SomeDelegate d = SomeMethod1`;  
  
`d += SomeMethod2;`

Invoking d will now call ***both*** `SomeMethod1` and `SomeMethod2`. Delegates are invoked in the `order` in `which they are added`.

The `-` and `-=` operators remove the right delegate operand from the left delegate
operand  
  
`d -= SomeMethod1;`  *//Invoking d will now cause only SomeMethod2 to be invoked.*

<div dir="rtl" style="width:80%; margin:auto;">
اگر دلیگیت دارای نوع بازگشتی غیر از `void` باشد، فقط مقدار بازگشتی متد آخری که فراخوانی شده در نهایت به کالر برگردانده می‌شود. مقادیر بازگشتی سایر متدها نادیده گرفته می‌شوند، که این ممکن است در برخی موارد مهم باشد.
</div>

In [6]:
public delegate int Calculator();
static int Add()
{
    Console.WriteLine("Adding");
    return 10;
}
static int Multiply()
{
    Console.WriteLine("Multiplying");
    return 20;
}
static int Subtract()
{
    Console.WriteLine("Subtracting");
    return 5;
}

Calculator calc = Add; // اضافه کردن متد Add به دلیگیت
calc += Multiply;      // اضافه کردن متد Multiply
calc += Subtract;      // اضافه کردن متد Subtract

int result = calc();   // فراخوانی دلیگیت و دریافت مقدار بازگشتی از آخرین متد فراخوانی شده
Console.WriteLine("Result: " + result); // نمایش نتیجه


Adding
Multiplying
Subtracting
Result: 5


### Generic Delegate Types

In [1]:
//A delegate type can contain generic type parameters:
public delegate T Transformer<T> (T arg);

public class Util
{
    public static void Transform<T> (T[] values, Transformer<T> t)
    {
        for (int i = 0; i < values.Length; i++)
            values[i] = t (values[i]);
    }
}

int Square (int x) => x * x;

int[] values = { 1, 2, 3 };
Util.Transform (values, Square); // Hook in Square

foreach (int i in values)
    Console.Write (i + " "); // 1 4 9

string Repeat(string s) => s += " "+s;
string[] names = {"ali","ardalan","taghi"};
Util.Transform (names, Repeat);

foreach (string n in names)
    Console.Write (n + " "); 

1 4 9 ali ali ardalan ardalan taghi taghi 

### The Func and Action Delegates

With ***generic delegates***, it becomes possible to write a `small set of delegate` types that are so general they can `work for methods of any return type` and any (reasonable) `number of arguments`. These delegates are the `Func` and `Action` delegates

In [ ]:
delegate TResult Func <out TResult> ();
delegate TResult Func <in T, out TResult> (T arg);
delegate TResult Func <in T1, in T2, out TResult> (T1 arg1, T2 arg2);
//... and so on, up to T16

delegate void Action ();
delegate void Action <in T> (T arg);
delegate void Action <in T1, in T2> (T1 arg1, T2 arg2);
//... and so on, up to T16

In [ ]:
public static void Transform<T> (T[] values, Func<T,T> transformer)
{
    for (int i = 0; i < values.Length; i++)
        values[i] = transformer (values[i]);
}

When C# was first introduced, the Func and Action delegates
did not exist (because generics did not exist). It’s for this historical
reason that much of .NET uses custom delegate types
rather than Func and Action.

### Delegate Compatibility

In [ ]:
//Delegate types are all incompatible with one another, 
// even if their signatures are the same:

In [ ]:
delegate void D1();
delegate void D2();
void Method1() { }

D1 d1 = Method1;
D2 d2 = d1; //نمی توانند به هم assign شوند

//we can do this
D2 d22 = new D2 (d1);

In [ ]:
//Delegate instances are considered equal if they have the same method targets
void Method1() { }
delegate void D();

D d1 = Method1;
D d2 = Method1;
Console.WriteLine (d1 == d2); // True

//Multicast delegates are considered equal,
//if they reference the same methods in the same order.

#### Parameter compatibility

When you call a `method`, you can supply arguments that have `more specific` types
than the parameters of that method. This is ordinary ***polymorphic*** behavior.

For the same reason, a `delegate` can have `more specific parameter` types than its method target. This is called ***contravariance***.

In [ ]:
delegate void StringAction (string s);
void ActOnObject (object o) => Console.WriteLine (o); // hello

StringAction sa = new StringAction (ActOnObject);
sa ("hello");

### Return type compatibility

If you call a `method`, you might get back a type that is `more specific` than what you asked for. This is ordinary `polymorphic` behavior.

In [ ]:
object RetriveString() => "hello";

string obj = RetriveString(); //use must get back more specific.

For the same reason, a `delegate’s target method` might return a `more specific type` than `described by the delegate`. This is called ***covariance***

In [ ]:
delegate object ObjectRetriever();

string RetriveString() => "hello";

ObjectRetriever o = new ObjectRetriever (RetriveString);
object result = o();
Console.WriteLine (result); // hello

#### Generic delegate type parameter variance

If you’re defining a generic delegate type, it’s good practice to do the following:  
- Mark a type parameter used only on the return value as ***covariant*** (`out`).
- Mark any type parameters used only on parameters as ***contravariant*** (`in`).

In [ ]:
delegate TResult Func<out TResult>(); //covariant

//This allows:
string GetString()=> "";
Func<string> x = new Func<string>(GetString);
Func<object> y = x;


delegate void Action<in T> (T arg);// contravarinat

void PrintObj(object obj){Console.WriteLine(obj.ToString());}

Action<object> xx = new Action<object>(PrintObj);
Action<string> yy = xx;